In [1]:
import azureml.core
from azureml.core import Workspace, Model

# Connect to the Workspace

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.13.0 to work with jbWorkspace


## Deploy the model for inference

In [3]:
model = ws.models['quiddler_model']
print(f'Model: {model.name}, Version: {model.version}')

Model: quiddler_model, Version: 1


In [19]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("icevision[all]==0.1.4")
myenv.add_pip_package("inference-schema")
myenv.add_pip_package("pygtrie")
myenv.save('source/inference-env.yml')

'source/inference-env.yml'

In [20]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig

In [ ]:
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = 'source',
                                   entry_script="score.py",
                                   conda_file="inference-env.yml")

deployment_config = LocalWebservice.deploy_configuration()

service_name = "quiddler-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

## Call the inference service

In [34]:
service.reload(wait=True)

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
Checking container health...
Local webservice is running at http://localhost:32795


In [35]:
endpoint = service.scoring_uri
print(endpoint)

http://localhost:32795/score


In [39]:
# JSON DATA
import base64
import requests
import json 

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

input_json = json.dumps({
    "n_cards": 5,
    "hand": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/test/IMG_4903.jpg'),
    "deck": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/train/A_1.jpg')
})
headers = { 'Content-Type':'application/json' }
predictions = requests.post(endpoint, input_json, headers = headers)
p = predictions.json()
print(str(p)+'\n')
print(f'Hand:     {p[0]}')
print(f'Deck:     {p[1]}')
if len(p) > 2:
    play = p[2][0]
    print(f'Score:    {play[0]}')
    print(f'Complete: {play[1]}')
    print(f'Words:    {[c[0] for c in play[2]]}')
    print(f'Pick up:  {p[2][1]}')
    print(f'Drop:     {p[2][2]}\n')
else:
    print('No possible play for these cards')    

['g/th/in/p/u', 'a', [[32, True, [['g/u/p', [16, 3, ['g', 'u', 'p']]], ['th/in', [16, 2, ['th', 'in']]]]], 'a', 'a']]

Hand:     g/th/in/p/u
Deck:     a
Score:    32
Complete: True
Words:    ['g/u/p', 'th/in']
Pick up:  a
Drop:     a



In [ ]:
print(service.get_logs())

## Shut it down

In [17]:
service.delete()
print("Service deleted.")

Container (name:charming_kilby, id:4fb08ec1fab48473dae41dbaf136485c149141070ee1117e46ceb74c621c7db6) cannot be killed.
Container has been successfully cleaned up.
Service deleted.


In [ ]:
!docker container ls

# Deploy into AKS

In [ ]:
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = 'source',
                                   entry_script="score.py",
                                   conda_file="inference-env.yml")
service_name = "quiddler-service2"

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

aks_target = AksCompute(ws,"jb-inf")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

In [ ]:
primary, secondary = service.get_keys()

In [ ]:
service.scoring_uri

In [ ]:
import requests
import json
import base64

# URL for the web service
endpoint = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

b64img = get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/test/IMG_4907.jpg')
input_json = json.dumps({"n_cards": 11, "hand": b64img})
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.json())

In [ ]:
service.delete()